In [7]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv( "../data/casas.csv")

In [9]:
def prepare( df ):
    
    le = preprocessing.LabelEncoder()
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    df["Ages"] = le.fit_transform( df["Ages"] )
    df1 = pd.get_dummies( df["Ages"] )
    
    
    le = preprocessing.LabelEncoder()
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    df["Condition"] = le.fit_transform( df["Condition"] )
    df2 = pd.get_dummies( df["Condition"] )
    
    le = preprocessing.LabelEncoder()
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    df["Location3"] = le.fit_transform( df["Location3"] )
    df3 = pd.get_dummies( df["Location3"])
    
    
    le = preprocessing.LabelEncoder()
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    df["FloorId"] = le.fit_transform( df["FloorId"] )
    df4 = pd.get_dummies( df["FloorId"])
    
    
    le = preprocessing.LabelEncoder()
    enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
    df["Stratum"] = le.fit_transform( df["Stratum"] )
    df5 = pd.get_dummies( df["Stratum"])
    
    
    ls = [ df1 , df2 , df3 , df4 , df5 , df["Area"] , df["Baths"]  , df["Rooms"] ]
    
    X = pd.concat( ls , axis = 1 )
    y = np.log( df["FormatedPrice"]/1e6 )
    
    return X.values , y.values

In [10]:
X , y = prepare( df )

In [11]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.8
num_boost_round = 1000
params = {"objective": "regression",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "metric":["mse"] , 
          "num_leaves": num_leaves,
           "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9 , 
}

In [ ]:
NFOLDS = 5
kfold = KFold(n_splits=NFOLDS, shuffle=True, random_state=218)


final_cv_train = np.zeros(len(y))
x_score = []

for s in range(16):
    cv_train = np.zeros(len(y))
    params['seed'] = s
    
    if True:
        kf = kfold.split( X ,   y )

        best_trees = []
        fold_scores = []
        
        for i, (train_fold, validate) in enumerate(kf):
            X_train, X_validate, label_train, label_validate = X[train_fold, :], X[validate, :], y[train_fold], y[validate]
            dtrain = lgb.Dataset(X_train, label_train )
            dvalid = lgb.Dataset(X_validate, label_validate, reference=dtrain )
            bst = lgb.train(params, dtrain, num_boost_round, valid_sets=dvalid , verbose_eval=100,early_stopping_rounds=100)
            best_trees.append(bst.best_iteration)
            cv_train[validate] += bst.predict(X_validate)
            
            
            score = mean_squared_error( label_validate, cv_train[validate] )
            print( score )
            fold_scores.append(score)

        final_cv_train += cv_train

        print("cv score:")
        print( mean_squared_error(y, cv_train))
        print( "current score:", mean_squared_error( y , final_cv_train / (s + 1.)), s+1)
        print(fold_scores)
        print(best_trees, np.mean(best_trees))

        x_score.append(mean_squared_error( y , cv_train))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.112072
[200]	valid_0's l2: 0.109472
[300]	valid_0's l2: 0.109834
Early stopping, best iteration is:
[207]	valid_0's l2: 0.109263
0.10926253215873803
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.205625
[200]	valid_0's l2: 0.203557
Early stopping, best iteration is:
[199]	valid_0's l2: 0.203544
0.20354428086017234
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.0965915
[200]	valid_0's l2: 0.095435
Early stopping, best iteration is:
[165]	valid_0's l2: 0.0953133
0.09531332351519167
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.146169
[200]	valid_0's l2: 0.143836
[300]	valid_0's l2: 0.143577
Early stopping, best iteration is:
[241]	valid_0's l2: 0.143399
0.1433988643482287
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.164236
[200]	valid_0's l2: 0.160596
